In [ ]:
# Google Colab Setup / Google Colab セットアップ
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    import sys
    from pathlib import Path
    
    proj_root = Path("/content/drive/MyDrive/image_processing_tutorial-main")
    if str(proj_root) not in sys.path:
        sys.path.insert(0, str(proj_root))
    
    print("Google Colab environment detected and configured!")
    print("Google Colab環境が検出され、設定されました！")
else:
    print("Local Jupyter environment detected")
    print("ローカルJupyter環境が検出されました")

# MRF Analysis Tutorial / MRF解析チュートリアル

This notebook demonstrates Markov Random Field (MRF) analysis for 1D image restoration, including parameter optimization and comparison of estimation methods.

このノートブックでは、パラメータ最適化と推定手法の比較を含む、1次元画像修復のためのマルコフ確率場（MRF）解析を実演します。

1. 1D Image Data Generation / 1次元画像データ生成
2. MRF Model Theory / MRFモデル理論
3. Matrix-based Estimation / 行列ベース推定
4. Gradient-based Estimation / 勾配ベース推定
5. Lambda Parameter Analysis / λパラメータ解析
6. Comparative Analysis / 比較解析

In [ ]:
# Import required libraries / 必要なライブラリをインポート
import numpy as np
import matplotlib.pyplot as plt
from utils import Image1D, MatrixEstimator, GradientEstimator
import time
from scipy.optimize import minimize_scalar

np.random.seed(42)

plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

## 1. 1D Image Data Generation / 1次元画像データ生成

We'll generate 1D image data using a random walk model and add noise to simulate real-world conditions.

ランダムウォークモデルを使用して1次元画像データを生成し、実世界の条件をシミュレートするためにノイズを追加します。

In [ ]:
# Parameters for image generation
N = 100  # Image length / 画像長
a = 1.0  # Variance parameter for true image / 真の画像の分散パラメータ
b = 0.5  # Variance parameter for noise / ノイズの分散パラメータ
seed = 42  # Random seed / ランダムシード

image_generator = Image1D(N)

u_true = image_generator.generate(a, seed=seed)

v_observed = image_generator.add_noise(u_true, b, seed=seed)

print("Image Generation Results / 画像生成結果")
print("=" * 50)
print(f"Image length: {N} pixels / 画像長: {N} ピクセル")
print(f"True image variance parameter (a): {a}")
print(f"Noise variance parameter (b): {b}")
print(f"Signal-to-noise ratio: {np.var(u_true)/np.var(v_observed - u_true):.2f}")
print(f"True image range: [{u_true.min():.2f}, {u_true.max():.2f}]")
print(f"Observed image range: [{v_observed.min():.2f}, {v_observed.max():.2f}]")

plt.figure(figsize=(15, 8))

plt.subplot(2, 2, 1)
plt.plot(u_true, 'b-', linewidth=2, label='True Image')
plt.plot(v_observed, 'r--', alpha=0.7, label='Observed Image (with noise)')
plt.xlabel('Pixel Index')
plt.ylabel('Intensity')
plt.title('1D Image Data')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
noise = v_observed - u_true
plt.plot(noise, 'g-', alpha=0.7)
plt.xlabel('Pixel Index')
plt.ylabel('Noise Level')
plt.title('Additive Noise')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()